## Yelp API

#### Create an app for free and get credentials on this website: https://www.yelp.com/developers/documentation/v3/authentication

#### Libraries

In [1]:
import requests
import json
from pprint import pprint

#### Support functions

In [2]:
def is_string(value):
    return isinstance(value, str)   
def is_int(value):
    return isinstance(value, int) 
def is_positive_int(value):
    return is_int(value) and value >= 0

def convert_num_to_date(yyyy, mm, dd):
    yyyy_str = str(yyyy).rjust(4, '0')
    mm_str = str(mm).rjust(2, '0')
    dd_str = str(dd).rjust(2, '0')
    
    return yyyy_str + '-' + mm_str + '-' + dd_str

#### Get establishments using Yelp API

In [3]:
def near_msc(s, d=1000, key='ybspQHnx1bPGeG73oLCcTRE_zZY3dWVwbvUq7xshW1XqjrhcDjPe-6j6S1TvXAgIdFXSdJcb8kV3aIDG9Sf5MYlqx2X2vFzJIKK4KH1CQh9DWOCVTMwEZV-vBk2_YHYx'):
    '''
        Function takes the search string "s", distance "d" in meters and API key, and returns the Yelp aliases 
        of all of the establishments matching the search string that are d-meters away 
        from (1300 University Ave, Madison WI), using given API key
    '''
    
    if not is_string(s):
        raise TypeError('Search should be a type of string')
        
    if not is_positive_int(d):
        raise TypeError('Distance should be a positive integer')
    
    if not is_string(key):
        raise TypeError('Key should be a type of string')
        
        
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization' : 'Bearer %s' % key}
    url_params = { 'term' : s,
                   'radius': d,
                   'location': '1300 University Ave, Madison WI'}
    
    r = requests.get(url, headers=headers, params=url_params)
    
    return [ res['alias'] for res in r.json()['businesses'] ]
    
near_msc('pub')

['jordans-big-ten-pub-madison',
 'sweet-home-wisconsin-madison',
 'the-brat-stand-madison',
 'the-library-cafe-and-bar-madison',
 'luckys-1313-brew-pub-madison-2',
 'the-sett-madison',
 'buffalo-wild-wings-madison-3',
 'greenbush-bar-madison']

#### Get best establishment using Yelp API

In [4]:
def near_msc(s, d=1000, key='ybspQHnx1bPGeG73oLCcTRE_zZY3dWVwbvUq7xshW1XqjrhcDjPe-6j6S1TvXAgIdFXSdJcb8kV3aIDG9Sf5MYlqx2X2vFzJIKK4KH1CQh9DWOCVTMwEZV-vBk2_YHYx'):
    '''
        Function takes the search string "s", distance "d" in meters and API key, and returns Yelp establishment 
        with the best raiting, matching the search string that are d-meters away 
        from (1300 University Ave, Madison WI), using given API key
    '''
    
    if not is_string(s):
        raise TypeError('Search should be a type of string')
        
    if not is_positive_int(d):
        raise TypeError('Distance should be a positive integer')
    
    if not is_string(key):
        raise TypeError('Key should be a type of string')
        
        
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization' : 'Bearer %s' % key}
    url_params = { 'term' : s,
                   'radius': d,
                   'location': '1300 University Ave, Madison WI'}
    
    r = requests.get(url, headers=headers, params=url_params)
    
    # find the best establishment
    best_rest_name = ''
    best_rest_raiting = -1
    
    for res in r.json()['businesses']:
        if best_rest_raiting < res['rating']:
            best_rest_name = res['alias']
            best_rest_raiting = res['rating']
    
    return {best_rest_name: best_rest_raiting}
    
near_msc('coffee')

{'indie-coffee-madison': 4.5}

## NASA API

#### Create an app for free and get credentials on this website: https://api.nasa.gov/

#### Get Near Earth Objects between dates

In [5]:
def get_neos_response(start_date, end_date, key='5uYddc7j4wUGPfBqw6K516jzzVvr6zcbV0zt1jet'):
    '''
        Function takes start_date and end_date (in format of YYYY-MM-DD) and NASA API key and returns
        Near Earth Objects based on the date of their closest approach to Earth
    '''
    
    if not is_string(start_date):
        raise TypeError('Start date should be a type of string')
    
    if not is_string(end_date):
        raise TypeError('End date should be a type of string')
    
    if not is_string(key):
        raise TypeError('Key should be a type of string')
        
        
    url = 'https://api.nasa.gov/neo/rest/v1/feed'
    url_params = { 'start_date' : start_date,
                   'end_date': end_date }
    
    r = requests.get(url, params=url_params, auth=(key, ''))

    return r.json()
    
neo_json = get_neos_response('2015-01-01', '2015-01-01')

In [6]:
print('Number of asteroids: ', neo_json['element_count'], '\n')

neo_attrs = [ item for item in neo_json['near_earth_objects']['2015-01-01'][0] ]
print('Attributes of the structure: \n', neo_attrs)

Number of asteroids:  8 

Attributes of the structure: 
 ['links', 'id', 'neo_reference_id', 'name', 'nasa_jpl_url', 'absolute_magnitude_h', 'estimated_diameter', 'is_potentially_hazardous_asteroid', 'close_approach_data', 'is_sentry_object']


#### Check date validity

In [7]:
def is_valid_date(yyyy, mm, dd):
    '''
        Function takes yyyy as a year, mm as a month and dd as a day and returns whether the actual date is valid
    '''
    
    month_to_day = { 1: 31,
                     2: 28,
                     3: 31,
                     4: 30,
                     5: 31,
                     6: 30,
                     7: 31,
                     8: 31,
                     9: 30,
                     10: 31,
                     11: 30,
                     12: 31 }
    
    # there is no data before the year 1900
    if yyyy < 1:
        return False
    
    # month should be between 1 and 12
    if mm < 1 or mm > 12:
        return False
    
    # day should be between 1 and max day of the month (considering the leap year in Feb)
    if dd < 1 or dd > month_to_day[mm]:
        # if it is a leap year -> return True
        if (mm == 2 and dd == 29) and (yyyy % 4 == 0) and (yyyy % 100 != 0 or yyyy % 400 == 0):
            return True
        
        return False

    return True
    
is_valid_date(1900, 2, 29)

False

#### Get Near Earth Objects on a specific date

In [8]:
def get_neos(yyyy, mm, dd):
    '''
        Function takes yyyy as a year, mm as a month and dd as a day and returns
        Near Earth Objects in the specified date of their closest approach to Earth
    '''
    
    if not is_positive_int(yyyy):
        raise TypeError('Year should be a positive integer')
    
    if not is_positive_int(mm):
        raise TypeError('Month should be a positive integer')
    
    if not is_positive_int(dd):
        raise TypeError('Day should be a positive integer')
        
    if not (is_valid_date(yyyy, mm, dd)):
        return None
        
    # convert to string and adjust the size
    date = convert_num_to_date(yyyy, mm, dd)

    return get_neos_response(date, date)
    
neo_json = get_neos(2029, 4, 13)

In [9]:
print('Number of asteroids: ', neo_json['element_count'])

Number of asteroids:  13


#### Get number of and average maximum diameter in km of all Near Earth Objects on a specific date

In [10]:
def get_neos_avg_maxdiam_km_count(yyyy, mm, dd):
    '''
        Function takes yyyy as a year, mm as a month and dd as a day and returns the number of and
        average maximum diameter in km of all Near Earth Objects on a specific date
    '''
    
    # get the response
    neo_json = get_neos(yyyy, mm, dd)
    
    # convert to string and adjust the size
    date = convert_num_to_date(yyyy, mm, dd)
    
    # calculate the average maximum diameter in km
    sum_of_maxdiam_km = sum([ el['estimated_diameter']['kilometers']['estimated_diameter_max'] 
                              for el in neo_json['near_earth_objects'][date]])

    return neo_json['element_count'], sum_of_maxdiam_km / neo_json['element_count']
    
avg_maxdiam_km = get_neos_avg_maxdiam_km_count(2020, 6, 8)
print('Number of asteroids: ', avg_maxdiam_km[0])
print('Average maximum diameter in km: ', avg_maxdiam_km[1])

Number of asteroids:  12
Average maximum diameter in km:  0.137111940625


#### Get info about the Apophis asteroid from the following article: https://www.npr.org/2021/03/27/981917655

In [11]:
neo_json = get_neos(2029, 4, 13)

In [12]:
print('Asteroid full name: ', neo_json['near_earth_objects']['2029-04-13'][0]['name'])

print('Max diameter in km: ', neo_json['near_earth_objects']['2029-04-13'][0]
      ['estimated_diameter']['kilometers']['estimated_diameter_max'])
print('Min diameter in km: ', neo_json['near_earth_objects']['2029-04-13'][0]
      ['estimated_diameter']['kilometers']['estimated_diameter_min'])

print('Closest distance in km: ', neo_json['near_earth_objects']['2029-04-13'][0]
      ['close_approach_data'][0]['miss_distance']['kilometers'])

Asteroid full name:  99942 Apophis (2004 MN4)
Max diameter in km:  0.6824015094
Min diameter in km:  0.3051792326
Closest distance in km:  38011.711742762
